In [27]:
# IMPORT DEPENDENCIES
from selenium import webdriver
from bs4 import BeautifulSoup as BS
import pandas as pd
import json

# Web Scraping Notes
For each state in states, pull relevant land data and save it to a custom mongo collection for that state. Do to the large amount of available listing, cap them to 100 per state. There are 25 per page, so simply visit 4 pages per state. \
\
Relevant data is:
* Address
    * Street number
    * City
    * State
    * zipcode
* Price
* Acreage
* Price per acre
* URL

### URL pattern = https://www.landwatch.com/{state}-land-for-sale/price-under-100000/acres-over-1/sort-price-low-high/page-{page_number}
### Listing class = d99b8 _7c2d9
### Price class = b04f6


# Testing with a single state

In [65]:
driver = webdriver.Firefox()

In [66]:
for page_num in range(1,5):
    print(page_num)

1
2
3
4


In [74]:
# Select single state
state = 'georgia'

#Initialize listings list
listings = []

# Iterate over desired number of pages
for page_num in range(1,5):
    
    # Build URL
    url = f"https://www.landwatch.com/{state}-land-for-sale/price-under-100000/acres-over-1/sort-price-low-high/page-{page_num}"
    
    # Visit Page
    driver.get(url)
    
    # Create soup object to simply search
    body = driver.find_element_by_tag_name("body")
    body_html = body.get_attribute("innerHTML")
    soup = BS(body_html)
    
    # Get listing elements
    li_elems = soup.find_all("div", class_="d99b8 _7c2d9")
    for li in li_elems:
        
        # Get address info from script tag
        script_dict = json.loads(li.find("script").text)
        # Address info
        address = script_dict['address']['streetAddress']
        city = script_dict['address']['addressLocality']
        zipcode = script_dict['address']['postalCode']
        
        # Get the link URL
        link = "https://www.landwatch.com" + li.a['href']
        
        # Retrieve the price
        price_str = li.find("div", class_="b04f6").text
        
        # Retrieve the acreage
        size_str = li.span.text.split(" acres")[0]
        
        # Getting the acreage and price can be a little finicky at times. 
        # Since I am only interested in listing where I can successfully retrieve these, 
        # the rest of the code is in a try/except statement.
        try:
            # Correct price formatting
            price = int(price_str.replace("$","").replace(",",""))

            # Correct size formatting
            size = float(size_str.replace(",",""))

            # Append data to listings
            listing = {
                "address": address,
                "city": city,
                "state": state,
                "zipcode": zipcode,
                "price": price,
                "size": size,
                "price/acre": price/size,
                "link": link
            }
            listings.append(listing)
        except:
            pass
        
listings_df = pd.DataFrame(listings)
print(f'Retrieved {len(listings_df)} for {state}.')
listings_df.head()

Retrieved 100 for georgia.


,address,city,state,zipcode,price,size,price/acre,link
0,Lot 105 Whitlow Lane,Lincolnton,georgia,30817,6800,1.84,3695.652174,https://www.landwatch.com/lincoln-county-count...
1,796 Woodbine Ln,Ranger,georgia,30630,9200,2.55,3607.843137,https://www.landwatch.com/gordon-county-county...
2,Indian Road,Lincolnton,georgia,30817,10000,1.18,8474.576271,https://www.landwatch.com/lincoln-county-count...
3,Indian Road,Lincolnton,georgia,30817,10000,1.22,8196.721311,https://www.landwatch.com/lincoln-county-count...
4,0 Pete Davis Rd,Newnan,georgia,30263,11950,1.00,11950.000000,https://www.landwatch.com/coweta-county-county...


In [69]:
listings_df

'TN'

In [80]:
for li in li_elems:
#     print(li)
#     print(li.find("script"))
#     print(li.find("script").text)
    
    script_dict = json.loads(li.find("script").text)
    address = script_dict['address']['streetAddress']
    city = script_dict['address']['addressLocality']
    state = script_dict['address']['addressRegion']
    zipcode = script_dict['address']['postalCode']
    print(address, city, state, zipcode)

222 Creekside Trail Forsyth GA 31029
0 Hawkeye Rabun Gap GA 30568
Dean Still Road Blackshear GA 31516
1008 Fleming Lake Court Lincolnton GA 30817
0 Northwood Trail Lot 37 Dudley GA 31022
Modoc Road Swainsboro GA 30401
Modoc Road Swainsboro GA 30401
 Modoc GA 30401
 Summertown GA 30401
Ben Hall Lake Rd East Dublin GA 31027
1235 Winding Ridge Drive Lincolnton GA 30817
1229 Winding Ridge Drive Lincolnton GA 30817
1217 Winding Ridge Drive Lincolnton GA 30817
1223 Winding Ridge Drive Lincolnton GA 30817
Marvin McEwen Rd Jackson GA 30233
0 Pinnacle Rd Clayton GA 30525
 Rising Fawn GA 30738
122 Creekside Trail Forsyth GA 31029
0 Orchard Rd Summerville GA 30747
 Hiawassee GA 30546
Coleman Chapel Bartow GA 30413
Hwy 25 Portal GA 30450
0 Fair Mountain Rabun Gap GA 30568
610 S Bell Street Harlem GA 30814
212 Northwood Trail Dudley GA 31022


In [75]:
driver.quit()

In [62]:
# Getting a list of states to use in script
tables = pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states')
state_col = tables[0].columns[0]
list(tables[0][state_col])

In [62]:
states = ['Alabama',
    'Alaska',
    'Arizona',
    'Arkansas',
    'California',
    'Colorado',
    'Connecticut',
    'Delaware',
    'Florida',
    'Georgia',
    'Hawaii',
    'Idaho',
    'Illinois',
    'Indiana',
    'Iowa',
    'Kansas',
    'Kentucky',
    'Louisiana',
    'Maine',
    'Maryland',
    'Massachusetts',
    'Michigan',
    'Minnesota',
    'Mississippi',
    'Missouri',
    'Montana',
    'Nebraska',
    'Nevada',
    'New Hampshire',
    'New Jersey',
    'New Mexico',
    'New York',
    'North Carolina',
    'North Dakota',
    'Ohio',
    'Oklahoma',
    'Oregon',
    'Pennsylvania',
    'Rhode Island',
    'South Carolina',
    'South Dakota',
    'Tennessee',
    'Texas',
    'Utah',
    'Vermont',
    'Virginia',
    'Washington',
    'West Virginia',
    'Wisconsin',
    'Wyoming']

len(states)

50

In [81]:
from pymongo import MongoClient as MC

In [82]:
client = MC()
DB = client.real_estate

In [111]:
states = []
for coll in DB.collection_names():
    df = pd.DataFrame(DB[coll].find())
    median = df['price/acre'].median()
    states.append({
        "state": coll,
        "median": median
    })
df = pd.DataFrame(states)
df

minimum = df['median'].min()

state = df['state'].loc[df['median'] == minimum]
list(state)[0]

'Texas'

In [105]:
df[df['median'] == minimum]

1    Texas
Name: state, dtype: object

In [86]:
DB.collection_names()

['Tennessee', 'Texas', 'Georgia', 'California']

In [93]:
texas_df = pd.DataFrame(DB.Texas.find())
texas_df['price/acre'].describe()

count      100.000000
mean      5306.816552
std       5970.541975
min        266.633333
25%        768.843914
50%       1820.048039
75%      10167.287978
max      20000.000000
Name: price/acre, dtype: float64

In [95]:
texas_df['price/acre'].median()

1820.0480392156865